# About torch grad

In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
torch.autograd.grad(loss, w, retain_graph=True)

In [ ]:
loss.backward(retain_graph=True)
print(w.grad)
print(b.grad)

# Mnist-classification Example

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# entering the directory
%cd /content/drive/MyDrive/ICSA_DLcourse/intro

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='mnist_data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='mnist_data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the neural network, loss function, and optimizer
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set up TensorBoard
writer = SummaryWriter('runs/mnist_classification')

In [ ]:
# Helper function to log images
def images_to_probs(images, labels, model):
    '''
    Generates predictions and their corresponding probabilities from a trained network
    '''
    output = model(images)
    # Convert output probabilities to predicted class (highest probability)
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds

def plot_classes_preds(images, labels, preds):
    '''
    Generates a Matplotlib Figure showing images and their predicted classes.
    '''
    fig = plt.figure(figsize=(12, 8))
    for idx in np.arange(6):
        ax = fig.add_subplot(2, 3, idx+1, xticks=[], yticks=[])
        img = images[idx] / 2 + 0.5  # unnormalize
        img = img.numpy()
        plt.imshow(np.transpose(img, (1, 2, 0)))
        ax.set_title(f'{preds[idx]} (true: {labels[idx].item()})',
                     color=('green' if preds[idx]==labels[idx].item() else 'red'))
    return fig

In [4]:
# Training function
def train(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
                writer.add_scalar('training_loss', running_loss / 100, epoch * len(train_loader) + batch_idx)
                running_loss = 0.0

        test_accuracy = test(epoch)
        writer.add_scalar('test_accuracy', test_accuracy, epoch)

# Testing function
def test(epoch):
    model.eval()
    correct = 0
    total = 0
    # Create an iterator for the test loader
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)
    
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

    # Log six test images and their predicted labels
    preds = images_to_probs(images, labels, model)
    fig = plot_classes_preds(images.cpu(), labels.cpu(), preds)
    writer.add_figure('test_predictions_vs_actuals', fig, global_step=epoch)
    
    return accuracy

# Train the model
train(num_epochs=5)

# Close the TensorBoard writer
writer.close()


Epoch [1/5], Step [100/938], Loss: 0.3257
Epoch [1/5], Step [200/938], Loss: 0.3824
Epoch [1/5], Step [300/938], Loss: 0.2819
Epoch [1/5], Step [400/938], Loss: 0.1822
Epoch [1/5], Step [500/938], Loss: 0.3453
Epoch [1/5], Step [600/938], Loss: 0.1572
Epoch [1/5], Step [700/938], Loss: 0.1590
Epoch [1/5], Step [800/938], Loss: 0.2946
Epoch [1/5], Step [900/938], Loss: 0.3167
Accuracy on test set: 92.19%
Epoch [2/5], Step [100/938], Loss: 0.1575
Epoch [2/5], Step [200/938], Loss: 0.1138
Epoch [2/5], Step [300/938], Loss: 0.2558
Epoch [2/5], Step [400/938], Loss: 0.1304
Epoch [2/5], Step [500/938], Loss: 0.1729
Epoch [2/5], Step [600/938], Loss: 0.1278
Epoch [2/5], Step [700/938], Loss: 0.1127
Epoch [2/5], Step [800/938], Loss: 0.2371
Epoch [2/5], Step [900/938], Loss: 0.1428
Accuracy on test set: 95.64%
Epoch [3/5], Step [100/938], Loss: 0.1903
Epoch [3/5], Step [200/938], Loss: 0.0750
Epoch [3/5], Step [300/938], Loss: 0.0462
Epoch [3/5], Step [400/938], Loss: 0.0747
Epoch [3/5], Step 